# 📚<span style = 'font-family:cursive;'> Libraries </span> 

In [ ]:
#!pip install efficientnet -q

In [ ]:
# Libraries
import sys
import warnings
warnings.filterwarnings('ignore')

sys.path = ['../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',] + sys.path


import glob
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import cv2
import random

# tensorflow
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from kaggle_datasets import KaggleDatasets

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics

import torch
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms


from tqdm import tqdm

from efficientnet_pytorch import model as enet

#albumentations
import albumentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau

# 📝<span style="font-family:cursive;"> Data Preparation</span>


In [ ]:
data_dir = '../input/seti-breakthrough-listen'
train_merger = os.path.join(data_dir,'train_labels.csv')
train_labels = pd.read_csv(train_merger)
print('train_label_csv : ' +str(train_labels.shape[0]))
#adding the path for each id for easier processing
train_labels['path'] = train_labels['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
train_labels.head()

In [ ]:
from kaggle_datasets import KaggleDatasets

data_dir ='../input/seti-breakthrough-listen'
path_dir = KaggleDatasets().get_gcs_path('seti-breakthrough-listen')

train = os.path.join(data_dir, 'train_labels.csv')
train_df = pd.read_csv(train)  
print("data_train_csv : " + str(train_df.shape[0]))
display(train_df.head(5))


sub = os.path.join(data_dir,'sample_submission.csv')
sub_df = pd.read_csv(sub)
print("data_submission_csv : " + str(sub_df.shape[0]) )
display(sub_df.head(5))

label_cols = sub_df.columns[1:]
#label_cols.values
labels = train_df[label_cols].values

# 📗<span style="font-family:cursive;"> Albumentations</span>

In [ ]:
from typing import *

class Transform:
    def __init__(self, aug_kwargs: Dict):
        albumentations_aug = [getattr(A, name)(**kwargs)
                            for name, kwargs in aug_kwargs.items()]
        albumentations_aug.append(ToTensorV2(p=1))
        self.transform = A.Compose(albumentations_aug)
    
    def __call__(self, image):
        image = self.transform(image = image)['image']
        return image

In [ ]:
class ModeTransform():
    def __init__(self, df_frame, config,type_mode,mode,target,type_spatial,type_channel,transform):
        self.df_frame = df_frame
        self.type_mode = type_mode
        self.config = config
        self.target = target
        self.file_names = df_frame['path'].values
        self.labels = df_frame['target'].values
        self.transform = transform
        self.mode = mode
        self.type_spatial = type_spatial
        self.type_channel = type_channel
        
    def __len__(self):
        return len(self.df_frame)

    def __getitem__(self, idx):
        image = np.load(self.file_names[idx])
        # print(image.shape) -> (6, 273, 256)
        
        if self.type_spatial:
            if self.type_mode == 'spatial_6ch':
                image = image.astype(np.float32)
                image = np.vstack(image) # no transpose here (1638, 256) 
                #image = np.vstack(image).transpose((1, 0))
                # print(image.shape) -> (256, 1638)
            elif self.type_mode == 'spatial_3ch':
                image = image[::2].astype(np.float32)
                image = np.vstack(image).transpose((1, 0))
        
        elif self.type_channel:
            if self.type_mode == '6_channel':
                image = image.astype(np.float32)
                image = np.transpose(image, (1,2,0))
            elif self.type_mode == '3_channel':
                image = image[::2].astype(np.float32)
                image = np.transpose(image, (1,2,0))
        
        if self.transform:
            image = self.transform(image)
        else:
            image = torch.from_numpy(image).float()

        if self.mode == 'test':
            return image    
        else:
            label = torch.tensor(self.labels[idx]).float()
            return image, label

In [ ]:
CONFIG = { 
    "TRAIN_TRANSFORMS": {        
        "VerticalFlip": {"p": 0.5},
        "HorizontalFlip": {"p": 0.5},
        "Resize": {"height": 640, "width": 640, "p": 1},
    }}
config = CONFIG

# Parameters
params_train  = {'mode'            : 'train',
                 'type_mode'       : 'spatial_6ch',
                 'target'          : True,
                 'type_spatial'    : True,
                 'type_channel'    : True}

train_dset = ModeTransform(train_labels,config,
                           **params_train,
                           transform=Transform(config["TRAIN_TRANSFORMS"]))

for i in range(2):
    image, label = train_dset[i]
    plt.imshow(image[0])
    plt.title(f'label: {label}')
    plt.show()
image.shape

In [ ]:
def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results 
     are the same every time we run.This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    return random_state

seed = 42
random_state = set_seed(seed)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('GPU is available')
else:
    device = torch.device('cpu')
    print('GPU not available, CPU used')

# 🗃 <span style="font-family:cursive;">Custom Dataset</span>

In [ ]:
class CustomDataset:
    
    def __init__(self, image_dir, targets, isTrain=True): 
        self.image_dir = image_dir
        self.targets = targets
        self.isTrain = isTrain

    def __len__(self):
        return len(self.image_dir)
    
    def __getitem__(self, idx):      
        image = np.load(self.image_dir[idx])
        image1 = np.vstack(image).transpose((1, 0)).astype(np.float32)[np.newaxis, ]
                
        return {
            "image_trnspose": torch.tensor(image1, dtype=torch.float),
            "targets": torch.tensor(self.targets[idx], dtype=torch.long),
        }   

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')
df.head()

# 🧪<span style="font-family:cursive;">Define Model</span>

In [ ]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))
        
        
        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=3, bias=False)

    def extract(self, x):
        return self.enet(x)
      
    def forward(self, x):
        x = self.conv1(x)
        x = self.extract(x)
        x = self.myfc(x)
        
        return x                               

# 🔄<span style="font-family:cursive;"> MixUp Augmentation</span>

In [ ]:
class MixUpAugmentation():
    '''Returns mixed inputs, pairs of targets, and lambda'''
    def __init__(self, x, y, alpha, use_cuda):
        self.x = x
        self.y = y
        self.alpha = alpha
        self.use_cuda = use_cuda
        
    def __getitem__(self):
        if self.alpha > 0:
            lmbda = np.random.beta(self.alpha, self.alpha)
        else:
            lmbda = 1
         
        batch_size = self.x.size()[0]
        if self.use_cuda:
            index = torch.randperm(batch_size).cuda()
        else:
            index = torch.randperm(batch_size)

        mixed_x = lmbda * self.x + (1 - lmbda) * self.x[index, :]
        y_a, y_b = self.y, self.y[index]
        return mixed_x, y_a, y_b, lmbda    

In [ ]:
class MixUpCriterion():
     def __init__(self, criterion, pred, y_a, y_b, lmbda):
        self.criterion = criterion
        self.pred = pred
        self.y_a = y_a
        self.y_b = y_b
        self.lmbda = lmbda
      
     def __getitem__(self):
        return self.lmbda * self.criterion(self.pred, self.y_a) + (1 - self.lmbda) * self.criterion(self.pred, self.y_b)    

In [ ]:
'''
Parameters
'''
params_MixUpAug  = {'alpha'            : 1.0,
                   'use_cuda'         : True} 

# 📑<span style="font-family:cursive;"> Train & Evaluate Loader</span>

In [ ]:
def train(data_loader, model, optimizer, device):
    
    model.train()
    
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        
        inputs1 = data["image_trnspose"]
        targets = data['targets']
        
        inputs1, targets_a, targets_b, lam = MixUpAugmentation(inputs1, 
                                                               targets.view(-1, 1), **params_MixUpAug)

        inputs1 = inputs1.to(device, dtype=torch.float)
        targets_a = targets_a.to(device, dtype=torch.float)
        targets_b = targets_b.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        outputs = model(inputs1)
        loss = MixUpCriterion(criterion, outputs, targets_a, targets_b, lam)
        loss.backward()
        optimizer.step()
        
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data["image_trnspose"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

In [ ]:
paths = [
 'efficientnet-b0-08094119.pth',
 'efficientnet-b1-dbc7070a.pth',
 'efficientnet-b2-27687264.pth',
 'efficientnet-b3-c8376fa2.pth',
 'efficientnet-b4-e116e8b3.pth',
 'efficientnet-b5-586e6cc6.pth',
 'efficientnet-b6-c76e70fd.pth',
 'efficientnet-b7-dcc49843.pth',
]
pretrained_model = {
    'efficientnet-b0': '../input/efficientnet-pytorch/' + paths[0],
    'efficientnet-b1': '../input/efficientnet-pytorch/' + paths[1],
    'efficientnet-b2': '../input/efficientnet-pytorch/' + paths[2],
    'efficientnet-b3': '../input/efficientnet-pytorch/' + paths[3],
    'efficientnet-b4': '../input/efficientnet-pytorch/' + paths[4],
    'efficientnet-b5': '../input/efficientnet-pytorch/' + paths[5],
    'efficientnet-b6': '../input/efficientnet-pytorch/' + paths[6],
    'efficientnet-b7': '../input/efficientnet-pytorch/' + paths[7],
}

In [ ]:
device = "cuda"
model_name = 'efficientnet-b4'
model = enetv2(model_name,out_dim = 1)
model.to(device)
model.load_state_dict(torch.load('../input/eff-mixup-v98/aug_training_0_10.pt'))

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}//{x}.npy')

In [ ]:
test_dataset = CustomDataset(image_dir=submission.img_path.values, 
                                     targets=submission.target.values,
                                      )

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=16, 
                                          shuffle=False, 
                                          num_workers=4)

test_predictions, test_targets = evaluate(test_loader,model, device)

In [ ]:
test_predictions = np.array(test_predictions)
submission.target = test_predictions[:, 0]
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)

# <span style="font-family:cursive;"> if it's notebook useful for you come on upvote 😀⬆🔝</span>